In [17]:
import random
import torch
#生成带噪声的人造数据集
#任务是使用有限样本的数据集来恢复该模型的参数
#使用低维数据 方便数据可视化
#生成2000个样本的数据，每个数据包含一个从正态分布采用的2个特征
#模型：Y=W * X(向量) + b + 噪声
def synthetic_data(w,b,num_examples):
    """Y=W * X(向量) + b + 噪声"""
    #num_examples表示生成的样本数量，len(w)表示每个样本的特征数。 num_examples行 len(w)列。
    X=torch.normal(0,1,(num_examples,len(w))) #均值为0 标准差1 len(w)每一组有几个变量 normal生成正态分布模型
    Y=torch.matmul(X,w)+b #矩阵乘法  2000*2 X 1*2 = 1*2000  matmul=matrix multiplication
    Y+=torch.normal(0,0.01,Y.shape) #均值0 标准差0.01的噪声 通过使用Y.shape作为参数，确保生成的随机数与Y具有相同的形状。
    return X,Y.reshape((-1,1)) #重新改变Y的形状为 任意行1列

true_w=torch.tensor([2,-3.4])
true_b=4.2
#features:每一行包含一个二维数据样本 labels:每一行包括一个一维标签
features,labels=synthetic_data(true_w,true_b,2000) #用真实的权重w和偏置b生成人工数据集2000个样本
print('features',features[0],'\nlabels',labels[0]) #有噪声

features tensor([ 0.6616, -1.0743]) 
labels tensor([9.1694])


In [18]:
# import matplotlib.pyplot as plt
# fig,ax=plt.subplots() 
# x_axis=features
# ep1=labels

    
# # ax.plot(x_axis,ep1) #图上画一些点并连接 折线图
# plt.plot(features[:,(1)],labels,1) #x,y,size
# plt.show() #输出画布

In [19]:
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #内核放挂掉
# # plt.scatter(features[:,(1)].detach().numpy(),labels.detach().numpy(),1)
# plt.scatter(features[:,(1)],labels,1) #x,y,size
# plt.show() #输出画布

In [20]:
#读取数据集：模型训练的时候需要对数据集进行遍历，每一次都取一个小批量样本，并且使用它们更新模型
#随机读取一个小批量样本

def data_iter(batch_size,featrues,labels):
    num_examples=len(featrues)
    indices=list(range(num_examples))
    #打乱列表顺序 实现随机读取
    random.shuffle(indices)
    #按batch_size读取数据集
    for i in range(0, num_examples, batch_size):
#         batch_indices = indices[i:min(i + batch_size, num_examples)]
#         batch_features = [features[j] for j in batch_indices]
#         batch_labels = [labels[j] for j in batch_indices]
#         yield batch_features, batch_labels
        batch_indices=torch.tensor(indices[i:min(i+batch_size,num_examples)]) #避免batch_size越界
        yield features[batch_indices],labels[batch_indices] #返回读取的数据 保存当前的位置 下次继续迭代
    
for batch_features, batch_labels in data_iter(10, features, labels):
    print("features:",batch_features, "\nlabels:",batch_labels)

features: tensor([[ 1.9901,  0.1008],
        [-0.7325,  0.0989],
        [ 1.6133,  0.5288],
        [-0.3622, -1.6471],
        [ 0.5295, -0.3756],
        [ 1.6006, -0.8179],
        [ 0.3063, -0.1169],
        [ 0.3627, -1.4525],
        [-0.6275,  0.8543],
        [ 0.0122,  0.3325]]) 
labels: tensor([[ 7.8342],
        [ 2.3959],
        [ 5.6230],
        [ 9.0750],
        [ 6.5470],
        [10.1577],
        [ 5.2277],
        [ 9.8633],
        [ 0.0365],
        [ 3.0781]])
features: tensor([[ 1.3032,  1.1514],
        [ 1.4945,  2.5098],
        [-1.0188,  0.1161],
        [ 0.1579,  0.4240],
        [-0.6330,  0.9218],
        [ 1.0477,  0.2136],
        [-0.6190, -1.8010],
        [ 0.4000, -0.7580],
        [-0.5022, -0.0308],
        [ 0.2015, -1.6676]]) 
labels: tensor([[ 2.8682],
        [-1.3350],
        [ 1.7743],
        [ 3.0847],
        [-0.1959],
        [ 5.5758],
        [ 9.0671],
        [ 7.5682],
        [ 3.3072],
        [10.2713]])
features: tensor([

features: tensor([[ 0.1219, -0.3482],
        [ 0.6567,  0.6743],
        [ 0.4281,  0.3846],
        [-1.4380,  0.5816],
        [-0.9063, -0.0295],
        [ 0.9860, -0.4931],
        [-1.0046,  0.3724],
        [ 0.1753, -1.2137],
        [ 1.2515, -0.0344],
        [-0.2558, -1.1172]]) 
labels: tensor([[ 5.6263],
        [ 3.2050],
        [ 3.7525],
        [-0.6455],
        [ 2.4929],
        [ 7.8441],
        [ 0.9261],
        [ 8.6648],
        [ 6.8258],
        [ 7.4849]])
features: tensor([[-0.3717, -0.4069],
        [ 1.7103,  0.0225],
        [-1.0330,  1.4452],
        [-0.6404,  0.4946],
        [ 0.6593, -1.0598],
        [ 1.1336, -1.1036],
        [-0.5166,  0.5449],
        [ 1.9608, -1.9607],
        [-0.4830, -0.8349],
        [ 1.4680, -1.3656]]) 
labels: tensor([[ 4.8498],
        [ 7.5487],
        [-2.7785],
        [ 1.2329],
        [ 9.1181],
        [10.2239],
        [ 1.3356],
        [14.7966],
        [ 6.1014],
        [11.7721]])
features: tensor([

labels: tensor([[ 5.7286],
        [ 2.7277],
        [ 7.5217],
        [ 1.6098],
        [-1.0476],
        [ 2.0042],
        [ 3.9429],
        [ 2.1075],
        [ 5.9598],
        [ 4.9741]])
features: tensor([[-1.8560,  1.3580],
        [-0.6107,  0.0232],
        [ 1.8090,  0.9391],
        [ 0.1947, -0.7048],
        [-1.2381,  0.4952],
        [-0.2244,  0.9866],
        [ 0.5055,  0.7950],
        [ 0.1040,  0.0882],
        [ 0.7799,  0.7373],
        [ 1.0283, -0.2930]]) 
labels: tensor([[-4.1322],
        [ 2.8894],
        [ 4.6381],
        [ 6.9811],
        [ 0.0334],
        [ 0.4140],
        [ 2.5075],
        [ 4.1054],
        [ 3.2607],
        [ 7.2600]])
features: tensor([[ 1.5188, -0.4095],
        [ 0.1340,  0.2393],
        [-0.8325,  0.2963],
        [-0.6734, -0.8219],
        [ 1.0534, -0.1511],
        [ 0.4832, -0.2265],
        [ 0.6277,  0.4023],
        [ 0.3213,  0.5998],
        [ 0.9885, -1.5044],
        [ 0.5351,  0.8911]]) 
labels: tensor([[ 

In [21]:
#划分训练集和测试集
#初始化模型的参数 在使用梯度下降算法优化之前，需要有一些参数，从正态分布中采样初始化权重，以及设置偏置为0
w=torch.normal(0,0.01,size=(2,1),requires_grad=True) # 维度 （2，1） 或（5，1）
b=torch.zeros(1,requires_grad=True) #尽可能接近true_w 和 true_b

#训练的任务是更新这些参数 直到这些参数足够拟合我们的数据，每次更新需要计算损失函数关于模型参数的梯度
#计算出梯度后 就向梯度最小的方向更新参数

#定义模型：将模型的输入和参数与模型输出关联起来
# 因为我们算的是线性模型，所以只需要计算输入的特征值X和模型权重w的矩阵向量乘积 然后加上偏差b
def linreg(X,w,b):
    """线性回归模型"""
    return torch.matmul(X,w)+b

#定义损失函数
def squared_loss(y_hat,y):
    """均方损失函数"""
    #y.reshape(y_hat.shape) 将y改造成y_hat一样的矩阵 才能加减
    return (y_hat-y.reshape(y_hat.shape))**2 / 2 

#定义优化算法：线性回归有解析解
#设置指定的训练次数，在每次训练中，从数据集随机取一个小批量，然后根据参数计算损失函数的梯度。
#然后，朝着减少损失的方向更新参数
def sgd(params,lr,batch_size):
    """小批量随机梯度下降""" 
    #设置学习率：lr(步长)， batch_size：批次大小
    #with :==try catch:不管是否出现异常 都自动回收资源
    #因为计算的损失是一个批量样本的总和，所以可以考虑使用批量大小为步长，刚好可以抵消
    #with torch.no_grad(): 是一个上下文管理器，用于控制是否计算并存储梯度信息。在这个上下文中，所有的操作都不会生成梯度(自动求导)，也不会进行梯度计算。
    with torch.no_grad(): 
        for param in params:
            #通过将参数的梯度乘以学习率，并除以批次大小，得到更新的大小，然后从参数的当前值中减去该更新大小。grad：求梯度(求导)
            param -= lr * param.grad / batch_size
            param.grad.zero_() #将参数的梯度清零，准备进行下一轮的梯度计算和更新。

#训练：每次读取一个小批量样本，通过模型获得一些预测值，计算损失，然后反向传播(向下传播),存储每个参数的梯度
#训练：重复：
#1，初始化参数
#2，重复训练：
#    2.1 计算梯度
#    2.2 更新参数

# 设置学习率和训练批次大小(超参数)
lr = 0.06
batch_size = 32
loss = squared_loss
net=linreg

# 训练模型
# 训练次数
num_epochs = 50
for epoch in range(num_epochs):
    #批量读取
    for X,Y in data_iter(batch_size,features,labels):
         # 前向传播：计算模型的预测值
        y_hat = net(X, w, b)
    
        l=loss(y_hat,Y) #计算XY的小批量损失值
        
        # 反向传播：计算损失函数关于模型参数的梯度
        l.mean().backward()
        # RuntimeError: grad can be implicitly created only for scalar outputs
        # 出现了非标量（non-scalar）损失函数的情况。PyTorch的自动微分机制要求梯度计算是基于标量的，因此无法为非标量张量创建梯度。选择 'mean' 或 'sum' 可以将损失降维为标量。
        
        #计算关于[w,b]的梯度,更新模型参数
        sgd([w,b],lr,batch_size)
        
        # 重置参数的梯度
        w.grad.zero_()
        b.grad.zero_()
        
    #展示训练精度
    with torch.no_grad():
        train_l=loss(net(features,w,b),labels)
        print(f"epoch {epoch+1},loss {float(train_l.mean()):f}")
        #随着训练次数增加 w和b越来越接近真实值
        for wi in w:
            print("w:",{wi.item()},end=' ')
        print(f"b:{b.item()}")
        #item将张量转化成标量输出
            
print(f"真实值：w:{true_w},b:{true_b}")

epoch 1,loss 13.242719
w: {0.23161225020885468} w: {-0.37117066979408264} b:0.48077642917633057
epoch 2,loss 10.413802
w: {0.4388643503189087} w: {-0.7015260457992554} b:0.9081789255142212
epoch 3,loss 8.193078
w: {0.6227522492408752} w: {-0.9946219325065613} b:1.2858134508132935
epoch 4,loss 6.447342
w: {0.7825323939323425} w: {-1.2569797039031982} b:1.6200302839279175
epoch 5,loss 5.069905
w: {0.9255500435829163} w: {-1.4907699823379517} b:1.916672706604004
epoch 6,loss 3.983219
w: {1.051613450050354} w: {-1.700760841369629} b:2.179708242416382
epoch 7,loss 3.134712
w: {1.1623750925064087} w: {-1.885774850845337} b:2.4114010334014893
epoch 8,loss 2.466794
w: {1.2601068019866943} w: {-2.050690174102783} b:2.6166961193084717
epoch 9,loss 1.940012
w: {1.3471009731292725} w: {-2.1983823776245117} b:2.798388719558716
epoch 10,loss 1.526423
w: {1.4233310222625732} w: {-2.329779863357544} b:2.959091901779175
epoch 11,loss 1.201187
w: {1.4905109405517578} w: {-2.446530818939209} b:3.10155010

In [22]:
# # 划分训练集和测试集 AI滴
# def train_test_split(X, Y, test_ratio=0.2):
#     """划分训练集和测试集"""
#     num_samples = X.shape[0]
#     num_test = int(num_samples * test_ratio)
#     indices = torch.randperm(num_samples)
#     test_indices = indices[:num_test]
#     train_indices = indices[num_test:]
#     X_train, X_test = X[train_indices], X[test_indices]
#     Y_train, Y_test = Y[train_indices], Y[test_indices]
#     return X_train, X_test, Y_train, Y_test

# # 划分训练集和测试集
# X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_ratio=0.2)

# # 设置学习率和训练批次大小
# lr = 0.10
# batch_size = 32

# # 训练模型
# num_epochs = 15
# for epoch in range(num_epochs):
#     # 在每个训练epoch中，随机生成小批量样本
#     indices = torch.randperm(X_train.shape[0])
#     for i in range(0, X_train.shape[0], batch_size):
#         batch_indices = indices[i:i+batch_size]
#         batch_X = X_train[batch_indices]
#         batch_Y = Y_train[batch_indices]

#         # 前向传播：计算模型的预测值
#         y_hat = linreg(batch_X, w, b)

#         # 计算损失函数
#         loss = squared_loss(y_hat, batch_Y)

#         # 反向传播：计算损失函数关于模型参数的梯度
#         loss.sum().backward()

#         # 更新模型参数
#         sgd([w, b], lr, batch_size)

#         # 重置参数的梯度
#         w.grad.zero_()
#         b.grad.zero_()
        
# print(f"w:{[w[0].item(),w[1].item()]},b:{b.item()}") #item将张量转化成标量输出
# print(f"真实值：w:{true_w},b:{true_b}")